In [42]:
import os
import numpy as np
import pandas as pd

In [43]:
# 設定 data_path
dir_data = '../data/'
f_app_train = os.path.join(dir_data, 'application_train.csv')
f_app_test = os.path.join(dir_data, 'application_test.csv')

app_train = pd.read_csv(f_app_train)
app_test = pd.read_csv(f_app_test)

檢視資料中各個欄位類型的數量

In [44]:
app_train.dtypes.value_counts()

float64    65
int64      41
object     16
dtype: int64

檢視資料中類別型欄位各自類別的數量

In [38]:
app_train.select_dtypes(include=["object"]).apply(pd.Series.nunique, axis = 0)

CODE_GENDER                    3
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
NAME_TYPE_SUITE                7
NAME_INCOME_TYPE               8
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
OCCUPATION_TYPE               18
WEEKDAY_APPR_PROCESS_START     7
ORGANIZATION_TYPE             58
FONDKAPREMONT_MODE             4
HOUSETYPE_MODE                 3
WALLSMATERIAL_MODE             7
EMERGENCYSTATE_MODE            2
dtype: int64

#### Label encoding
有仔細閱讀[參考資料](https://medium.com/@contactsunny/label-encoder-vs-one-hot-encoder-in-machine-learning-3fc273365621)的人可以發現，Label encoding 的表示方式會讓同一個欄位底下的類別之間有大小關係 (0<1<2<...)，所以在這裡我們只對有類別數量小於等於 2 的類別型欄位示範使用 Label encoding，但不表示這樣處理是最好的，一切取決於欄位本身的意義適合哪一種表示方法

In [46]:
from sklearn.preprocessing import LabelEncoder

In [59]:
# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
#print('%d columns were label encoded.' % le_count)
print(app_train)

        SK_ID_CURR  TARGET  NAME_CONTRACT_TYPE CODE_GENDER  FLAG_OWN_CAR  \
0           100002       1                   0           M             0   
1           100003       0                   0           F             0   
2           100004       0                   1           M             1   
3           100006       0                   0           F             0   
4           100007       0                   0           M             0   
5           100008       0                   0           M             0   
6           100009       0                   0           F             1   
7           100010       0                   0           M             1   
8           100011       0                   0           F             0   
9           100012       0                   1           M             0   
10          100014       0                   0           F             0   
11          100015       0                   0           F             0   
12          

#### One Hot encoding
pandas 中的 one hot encoding 非常方便，一行程式碼就搞定

In [32]:
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

print(app_train['CODE_GENDER_F'].head())
print(app_train['CODE_GENDER_M'].head())
print(app_train['NAME_EDUCATION_TYPE_Academic degree'].head())

0    0
1    1
2    0
3    1
4    0
Name: CODE_GENDER_F, dtype: uint8
0    1
1    0
2    1
3    0
4    1
Name: CODE_GENDER_M, dtype: uint8
0    0
1    0
2    0
3    0
4    0
Name: NAME_EDUCATION_TYPE_Academic degree, dtype: uint8


可以觀察到原來的類別型欄位都轉為 0/1 了

## 作業
將下列部分資料片段 sub_train 使用 One Hot encoding, 並觀察轉換前後的欄位數量 (使用 shape) 與欄位名稱 (使用 head) 變化

In [110]:
app_train = pd.read_csv(f_app_train)
sub_train = pd.DataFrame(app_train['WEEKDAY_APPR_PROCESS_START'])
print(sub_train.shape)
print(sub_train)

(307511, 1)
       WEEKDAY_APPR_PROCESS_START
0                       WEDNESDAY
1                          MONDAY
2                          MONDAY
3                       WEDNESDAY
4                        THURSDAY
5                       WEDNESDAY
6                          SUNDAY
7                          MONDAY
8                       WEDNESDAY
9                        THURSDAY
10                       SATURDAY
11                         FRIDAY
12                         FRIDAY
13                       THURSDAY
14                         MONDAY
15                       SATURDAY
16                       THURSDAY
17                         MONDAY
18                         FRIDAY
19                         MONDAY
20                         FRIDAY
21                         MONDAY
22                       THURSDAY
23                         FRIDAY
24                       THURSDAY
25                       SATURDAY
26                         MONDAY
27                       SATURDAY
28

In [117]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()


# 轉換 label encoding
# 先將data frame切片為list,再加入fit
le.fit(sub_train.iloc[:,0])

# 之後transform為labe encoding
le_result=le.transform(sub_train.iloc[:,0])
print(le.classes_)
print(le_result)


# 轉換成one hot encoding
ohe_result=pd.get_dummies(sub_train)
# 取代原本的column label
ohe_result.columns = le.classes_
print(ohe_result.head())



['FRIDAY' 'MONDAY' 'SATURDAY' 'SUNDAY' 'THURSDAY' 'TUESDAY' 'WEDNESDAY']
[6 1 1 ... 4 6 4]
   FRIDAY  MONDAY  SATURDAY  SUNDAY  THURSDAY  TUESDAY  WEDNESDAY
0       0       0         0       0         0        0          1
1       0       1         0       0         0        0          0
2       0       1         0       0         0        0          0
3       0       0         0       0         0        0          1
4       0       0         0       0         1        0          0
